capture audio and video data

In [2]:
import cv2 as cv
import pyaudio
import wave
import time
import threading

def record_video_and_audio():
    cap = cv.VideoCapture(0)
    fourcc = cv.VideoWriter_fourcc(*'mp4v')
    out = cv.VideoWriter('C:\\Users\\chels\\Desktop\\COSC490-MER\\outputs\\video\\output.mp4', fourcc, 20.0, (640, 480))

    CHUNK = 4096  # Increased chunk size for smoother audio
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 44100
    WAVE_OUTPUT_FILENAME = "C:\\Users\\chels\\Desktop\\COSC490-MER\\outputs\\audio\\output.wav"
    frames = []

    p = pyaudio.PyAudio()
    stream = None

    audio_started = False
    audio_running = True

    def audio_record():
        nonlocal stream, frames, audio_running
        stream = p.open(format=FORMAT,
                        channels=CHANNELS,
                        rate=RATE,
                        input=True,
                        frames_per_buffer=CHUNK)
        print("* recording audio")
        while audio_running:
            data = stream.read(CHUNK, exception_on_overflow=False)
            frames.append(data)

    audio_thread = threading.Thread(target=audio_record)
    audio_thread.start()

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break
        out.write(frame)
        cv.imshow('frame', frame)

        if cv.waitKey(1) == ord('q'):
            break

    # Cleanup video
    cap.release()
    out.release()
    cv.destroyAllWindows()

    # Cleanup audio
    audio_running = False
    audio_thread.join()
    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()
    print("* done recording audio")
record_video_and_audio()

* recording audio
* done recording audio


grab the transcript

In [3]:
import requests
import time

def transcribe_audio():
  base_url = "https://api.assemblyai.com"

  headers = {
      "authorization": "6a35340cac1c443e8e4bbc1d027a3ad5"
  }
  with open("outputs/audio/output.wav", "rb") as f:
    response = requests.post(base_url + "/v2/upload",
                            headers=headers,
                            data=f)

  audio_url = response.json()["upload_url"]


  data = {
      "audio_url": audio_url,
      "speech_model": "universal"
  }

  url = base_url + "/v2/transcript"
  response = requests.post(url, json=data, headers=headers)

  transcript_id = response.json()['id']
  polling_endpoint = base_url + "/v2/transcript/" + transcript_id

  while True:
    transcription_result = requests.get(polling_endpoint, headers=headers).json()
    transcript_text = transcription_result['text']

    if transcription_result['status'] == 'completed':
      print("Transcription completed.")
      return transcript_text
      # with open("transcript.txt", "w") as file:
      #   file.write(transcript_text) # we don't necessarily need to write it into a file
      # file.close()

    elif transcription_result['status'] == 'error':
      raise RuntimeError(f"Transcription failed: {transcription_result['error']}")

    else:
      time.sleep(3)
text = transcribe_audio()

Transcription completed.


In [4]:
text

"I feel really bad because today I accidentally bumped into somebody without saying excuse me, and I feel like they probably don't like me now because I didn't say excuse me to them. And I feel really bad, and I don't know what to do to resolve the situation. It's been weighing on my mind really heavily today because I kind of had a rough day, and I feel like that just made it a little bit worse, and I think that they probably don't like me now."

sentiment analysis

In [5]:
import nltk
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
analyzer = SentimentIntensityAnalyzer()

def tokens_to_text(tokens):
    text = " ".join(tokens)
    score = analyzer.polarity_scores(text)
    return score

def determine_sentiment(score):
    if score['compound'] >= 0.05:
        return 'positive'
    elif score['compound'] <= -0.05:
        return 'negative'
    else:
        return 'neutral'
    
score = analyzer.polarity_scores(text)
sentiment = determine_sentiment(score)

In [6]:
sentiment

'negative'

video analysis

In [14]:
from fer.fer import FER
from fer.classes import Video
import pandas as pd
def analyze_video_emotions():
    emotion_detector = FER(mtcnn=True)
    video = Video("C:\\Users\\chels\\Desktop\\COSC490-MER\\outputs\\video\\output.mp4")


    emotions = video.analyze(emotion_detector, display=False, frequency=5)
    df = pd.DataFrame(emotions)


    stats = df.describe()
    return stats.loc['mean'].nlargest(2).index.tolist()


In [ ]:

emotions = analyze_video_emotions()


INFO:fer:20.00 fps, 877 frames, 43.85 seconds
INFO:fer:Making directories at output
INFO:fer:Async I/O enabled for frame saving
176frames [00:30,  5.85frames/s]                     
INFO:fer:Waiting for async I/O to complete...
INFO:fer:Async I/O completed
INFO:fer:Completed analysis: saved to output\output_output.mp4
INFO:fer:Starting to Zip
INFO:fer:Compressing: 28%
INFO:fer:Compressing: 56%
INFO:fer:Compressing: 85%
INFO:fer:Zip has finished


In [17]:

emotion1, emotion2 = emotions[0], emotions[1]

In [18]:
emotion1, emotion2

('sad1', 'fear0')

therapyAI chatbot response

In [19]:
from ollama import chat
from ollama import ChatResponse
response: ChatResponse = chat(model='gemma3', messages=[
  {
    'role': 'system',
    'content': f'Based on the user\'s two primary emotions {emotion1} and {emotion2} from the video analysis, and based on the sentiment of the text which is {sentiment}, generate a therapuetic, empathetic response to the user\'s words. Avoid asking follow-up questions (if waranted), but rather provide advice as a therapist would. The user said: {text}'
  }
])
# or access fields directly from the response object
print(response.message.content)

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


It sounds like you’re carrying a really heavy burden of self-doubt and worry right now, and it’s completely understandable why you feel so intensely bad. The fact that you’re experiencing this significant discomfort – this feeling of not saying ‘excuse me’ and then imagining the worst – speaks volumes about your inherent desire to be considerate and polite. 

Let's acknowledge that your anxiety is understandably amplified by a tough day, and that’s creating a feedback loop where you’re focusing on a single, relatively minor event and letting it magnify into something much more significant.  It’s incredibly common to do this, especially when we're already feeling vulnerable. 

What you’re feeling – this worry that someone dislikes you – isn’t necessarily based on reality. People are often far more forgiving than we give them credit for. More importantly, focusing solely on this one instance is preventing you from enjoying the rest of your day. 

Try this: Gently remind yourself that you

final call

In [ ]:
record_video_and_audio() # record and capture the audio
